# MovieLens

[MovieLens](http://www.movielens.org/) is a website where users can submit ratings for movies that they watch and receive recommendations for other movies they might enjoy. The data is collected and made publicly available for research. We will be working with a data set of 1 million user ratings of movies. You can find this data set and even larger ones at http://grouplens.org/datasets/movielens/.

## Reading in the Data

Note that the data consists of three data frames: one with information about the users, another containing the ratings, and yet another with information about the movies. See the readme file (/data/movielens/README) for more information.

In [1]:
import pandas as pd

unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('/data/movielens/users.dat', sep='::', header=None,
                      names=unames, engine="python")

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('/data/movielens/ratings.dat', sep='::', header=None,
                        names=rnames, engine="python")

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('/data/movielens/movies.dat', sep='::', header=None,
                       names=mnames, engine="python")

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## Question 1

Develop a way to rank movies. Why is it not a good idea to simply sort the movies by average rating? (You may want to try calculating this first.) Then, explain your methodology and use it to produce a list of the Top 10 movies of all time.

In [244]:
from scipy.stats import t
ratings.head()
#movies.head()
#users.head()
moviejoinratings = pd.merge(ratings,movies, on = "movie_id")
moviejoinratings.head()
reviewers = moviejoinratings.groupby("title")["rating"] ## group ratings by movies
n = reviewers.count() ## how many reviewed each movie
mean = reviewers.mean() ## what was the mean review in each group.
std = reviewers.std() ## what was standard deviation in each group.
## lets organize the ratings based on their lower bound on a 95% CI. 
lower = mean - (t.ppf(.975,n) * std/(n**.5))

## I personally dont trust a movie review unless it has 100 or more reviews. lets restrict our results to 100 or more reviewers.
## If a moive is truly in the top 10 it must easily have 100 + reviews. 
top10 = lower[n >= 100].sort_values(ascending = False)
top10.head(10) ## the top 10 list

ratings.head()






,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


__You can't simply use the means because they are dependent on the number of users. This means there could be a movie in the data that had only one reivew ( or a mall number of reviews) that received a high rating because so few people reviewed it. In reality the movie could be terrible but its average rating may not reflect that because of such a small smample size. __
    
__I used the lower bound of a 95% CI as the measure of the ratings for movies. I did this because this will adjust for movie ratings with a small amount of reviews, For movies with small amount of reviews there small n valuse will produce a large margin of error and thus the lower bounds will tend to be very low. IN addition, I also restricted my top 10 movies to movies that only had 100 or more reviews. My reasoning behind this was to get rid of 'cult classics'. If a movie is trruly popular enough to be in the top ten that it will easily have more thatn 100 reviews. When I did this it took out the movie 'Sanjuro' that I had never heard of and Star wars Episode IV made the list as a result. If star wars epsiode IV isn't in the top 10 movies of all time and 'Sanjuro' is then clearly the top 10 list is wrong lol. __ 

## Question 2

Restrict to movies with at least 200 ratings. For each movie, calculate the difference between the average male rating and the average female rating. Based on these differences between average male ratings and average female ratings, what movies were the most male-friendly? What movies were the most female-friendly?

In [246]:
twohunnid = ratings.groupby("movie_id")['user_id'].count().reset_index() ## number of reviews per movie
ratings200 = twohunnid[twohunnid['user_id'] >= 200]
ratings200
ratings200 = pd.merge(ratings200, ratings, on = 'movie_id', how = 'inner', copy = False )
## going to rename the columns quickly:
columns = ratings200.columns.values
columns[1] = 'Total_Reviews'
columns[2] = 'user_id'
ratings200.columns = columns
###########
rategender = pd.merge(ratings200, users, on = "user_id")

avggend = rategender.groupby(['movie_id' , 'gender' ])["rating"].mean().reset_index()
avggend = avggend.pivot_table(index = 'movie_id', columns = 'gender')
avggend["Difference"] = avggend[('rating', 'M')] - avggend[('rating', 'F')]
avggend = pd.concat([avggend['Difference'], movies.set_index('movie_id')], axis = 1)
avggend.dropna(0).sort_values("Difference")





,Difference,title,genres
movie_id,,,
1088,-0.830782,Dirty Dancing (1987),Musical|Romance
203,-0.691567,"To Wong Foo, Thanks for Everything! Julie Newm...",Comedy
2468,-0.676359,Jumpin' Jack Flash (1986),Action|Comedy|Romance|Thriller
1380,-0.608224,Grease (1978),Comedy|Musical|Romance
879,-0.586447,"Relic, The (1997)",Horror
1021,-0.581662,Angels in the Outfield (1994),Children's|Comedy
261,-0.548849,Little Women (1994),Drama
...,...,...,...
1261,0.611985,Evil Dead II (Dead By Dawn) (1987),Action|Adventure|Comedy|Horror


__The most male friendly movie is the "Good, The Bad and The Ugly" this comes as no suprise as we tyupically think that men prefer more action based movies while females prefer romantic softer movioes. This is further illustrated by the fact that the most female friendly movie was Dirty Dancing. __  

## Question 3

Calculate the average rating by genre. Note that a movie can belong to multiple genres. You will have to write some code that parses the `genres` column of the `movies` table. What genre had the highest average rating? What genre had the lowest?

In [247]:

merge = pd.merge(movies, ratings, on= 'movie_id')
genres = movies['genres'].str.cat(sep = '|').split("|") ## farm all the genres from movies df.
genres = list(set(genres)) # put each unique genre in a list. 

for Mtype in genres:
    merge[Mtype] = merge['genres'].str.contains(Mtype)

avg = list()
type = list()
df = pd.DataFrame()
for Mtype in genres:
    type.append(Mtype)
    num = merge[Mtype].sum() ## docs
    docs = merge[merge[Mtype] == True]
    avgrate = docs['rating'].sum() / num
    avg.append(avgrate)
df['Genres'] = type
df['Average Rating'] = avg
df.sort_values('Average Rating')





,Genres,Average Rating
8,Horror,3.215013
12,Children's,3.422035
2,Fantasy,3.447371
4,Sci-Fi,3.466521
1,Adventure,3.477257
10,Action,3.491185
13,Comedy,3.522099
...,...,...
14,Mystery,3.668102
17,Animation,3.684868


__Film-Noir has the highest average rating while Horror ( NO suprise!) has the lowest average rating. __

## Question 4

Formulate a question of your own that you can answer using this MovieLens data. State clearly what your question is and what your findings are. Bonus points are available if you find something interesting!

**Tip:** You may find the `occupation` column of `users` to be a rich source of interesting questions. See the README file (/data/movielens/README) for information about how `occupation` is coded in the data.

__The first digit in a zip code is an indicator of what region a city is in within the united states. In total there are 10 regions in the united states numbere s 0-9. Using this fact, lets explore if the top rated movie is independent of region in the united states. The Regions are as follows (source: wikipedia) __


    0 = Connecticut (CT), Massachusetts (MA), Maine (ME), New Hampshire (NH), New Jersey (NJ), New York (NY, Fishers Island only), Puerto Rico (PR), Rhode Island (RI), Vermont (VT), Virgin Islands (VI), Army Post Office Europe (AE), Fleet Post Office Europe (AE)
    1 = Delaware (DE), New York (NY), Pennsylvania (PA)
    2 = District of Columbia (DC), Maryland (MD), North Carolina (NC), South Carolina (SC), Virginia (VA), West Virginia (WV)
    3 = Alabama (AL), Florida (FL), Georgia (GA), Mississippi (MS), Tennessee (TN), Army Post Office Americas (AA), Fleet Post Office Americas (AA)
    4 = Indiana (IN), Kentucky (KY), Michigan (MI), Ohio (OH)
    5 = Iowa (IA), Minnesota (MN), Montana (MT), North Dakota (ND), South Dakota (SD), Wisconsin (WI)
    6 = Illinois (IL), Kansas (KS), Missouri (MO), Nebraska (NE)
    7 = Arkansas (AR), Louisiana (LA), Oklahoma (OK), Texas (TX)
    8 = Arizona (AZ), Colorado (CO), Idaho (ID), New Mexico (NM), Nevada (NV), Utah (UT), Wyoming (WY)
    9 = Alaska (AK), American Samoa (AS), California (CA), Guam (GU), Hawaii (HI), Marshall Islands (MH), Federated States of Micronesia (FM), Northern Mariana Islands (MP), Oregon (OR), Palau (PW), Washington (WA), Army Post Office Pacific (AP), Fleet Post Office Pacific (AP)


In [248]:
zipcodes = users['zip'].str[:1] ## farm all the genres from movies df.
new_users = users
new_users['Region'] = zipcodes
new_users = new_users.merge(ratings, on = 'user_id')
new_users_mean = new_users.groupby(['movie_id', 'Region'])['rating'].mean().reset_index()
n = new_users.groupby(['movie_id', 'Region'])['rating'].count().reset_index()
new_users_mean = pd.merge(new_users_mean, movies, on = 'movie_id')
new_users_mean['total reviewers'] = n['rating']
new_users_mean = new_users_mean[new_users_mean['total reviewers'] >= 100]


tops = list()
regions = list()
title = list()
df = pd.DataFrame()
zips = list(set(zipcodes))
for z in zips:
    regions.append(z)
    bob = new_users_mean['Region'] == z
    tempusers = new_users_mean[bob]
    top = tempusers['rating'].max()
    tops.append(top)
df['Regions'] = regions
df['rating'] = tops
df = pd.merge(df, new_users_mean, on = 'rating', how = 'inner')
df.sort_values('Regions')

,Regions,rating,movie_id,Region,title,genres,total reviewers
8,0,4.666667,50,0,"Usual Suspects, The (1995)",Crime|Thriller,207
9,1,4.579365,858,1,"Godfather, The (1972)",Action|Crime|Drama,252
6,2,4.604317,50,2,"Usual Suspects, The (1995)",Crime|Thriller,139
2,3,4.629032,318,3,"Shawshank Redemption, The (1994)",Drama,124
1,4,4.655602,318,4,"Shawshank Redemption, The (1994)",Drama,241
5,5,4.598361,318,5,"Shawshank Redemption, The (1994)",Drama,244
4,6,4.670886,318,6,"Shawshank Redemption, The (1994)",Drama,158
0,7,4.680272,318,7,"Shawshank Redemption, The (1994)",Drama,147
3,8,4.531746,2762,8,"Sixth Sense, The (1999)",Thriller,126
7,9,4.627737,922,9,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Film-Noir,137


__ Interestingly enough, The top rated movie differs by region!, 50% of the regions still had shawshank redemption as their favorite movie, followd by 20% of the regions having the Usual suspects as their favorite movie. THe region including California (9) had Sunset BLVD as their favorite movies.... what a coincidence lol. 

## Submitting this Lab

Now, restart your kernel and re-run your entire notebook from beginning to end. Make sure there are no errors or bugs. When you have verified this, open the Terminal on JupyterHub and type 

`nbgrader submit Lab-04-14 --course dlsun`

to submit this lab.